In [26]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [33]:
ray_results = Path("~/ray_results").expanduser()
assert ray_results.is_dir()
last_experiment = max(ray_results.glob('_aux_train_daostack_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

def _aux_train_daostack(*args, **kwargs):
    raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), _aux_train_daostack)
rg = tuner.get_results()
rg.get_best_result()

2023-08-02 16:01:50,952	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-08-02 16:01:51,033	WARNING experiment_analysis.py:916 -- Failed to read the results for 1 trials:
- /home/daviddavo/ray_results/_aux_train_daostack_2023-08-02_15-52-56/_aux_train_daostack_9ba2bbbb_66_batch_size=8,conv_layers=5,embedding_dim=6,learning_rate=0.9851,max_epochs=50_2023-08-02_15-53-56


Loading results from /home/daviddavo/ray_results/_aux_train_daostack_2023-08-02_15-52-56...


ValueError: No metric is provided. Either pass in a `metric` arg to `get_best_result` or specify a metric in the `TuneConfig` of your `Tuner`.